* Imports and definitions

In [1]:
import plotly.graph_objects as go
from scipy import *
import numpy as np
import pywt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


samplingRate = 360  # Hz (given in the doc)

In [2]:
def draw(amplitudes, indices=None):
    if type(indices) == "list":  # if we have certain indices or not
        fig_cont = go.Figure()
        fig_cont.add_trace(
            go.Scatter(x=indices, y=amplitudes, mode="lines", name="Continuous Signal")
        )
    else:
        fig_cont = go.Figure()
        fig_cont.add_trace(
            go.Scatter(y=amplitudes, mode="lines", name="Continuous Signal")
        )
    fig_cont.update_layout(
        title="Continuous Signal",
        xaxis_title="Index",
        yaxis_title="Amplitude",
        width=1000,
        height=500,
    )

    fig_cont.show()  # For notebooks

In [3]:
# HB means HeartBeat
def readHB(fileName):
    signals = []
    with open(fileName) as l:
        line = l.readline()
        while line:
            signal = line.split("|")
            signals.append([float(value) for value in signal[:-1]])
            line = l.readline()
    return signals

In [4]:
# dirPath = "C:/Users/omart/Desktop/level 2/python/ECG-Classification/Data"
dirPath = "/kaggle/input/ecg-signals/Data"

In [5]:
healthySignals = readHB(f"{dirPath}/Normal_Train.txt")

In [6]:
PVCSignals = readHB(f"{dirPath}/PVC_Train.txt")

## After some investigating , I noticed the following:
* healthySignals and PVCSignals each contain 200 signals
* Each signal has 300 points
* Max amplitude is 1.6692
* Min amplitude is -2.1939

In [7]:
draw(healthySignals[104])  # drawing a random signal to see what we are working with

In [8]:
draw(PVCSignals[180])

### There's a huge difference between the healthy and the PVC , hopefully the classification won't be hard

### Applying the Band Pass filter with cutoffs = [0.5,40]

In [9]:
b, a = signal.butter(
    N=4, Wn=[0.5, 40], btype="bandpass", fs=samplingRate
)  # getting the filter coofecients

In [10]:
# filteredHealthy = [signal.lfilter(b, a, s) for s in healthySignals]
# filteredPVC = [signal.lfilter(b, a, s) for s in PVCSignals]
# filteredHealtyTest = [signal.lfilter(b, a, s) for s in readHB(f"{dirPath}/Normal_Test.txt")]
# filteredPVCTest = [signal.lfilter(b, a, s) for s in readHB(f"{dirPath}/PVC_Test.txt")]
filteredHealthy =  healthySignals
filteredPVC = PVCSignals
filteredHealtyTest =  readHB(f"{dirPath}/Normal_Test.txt")
filteredPVCTest = readHB(f"{dirPath}/PVC_Test.txt")

### The effect of filtering is very clear. Removed all unneeded noise

In [11]:
draw(filteredHealthy[104])

### Now will apply DWT and take the details coefficients because they contain the difference between normal and PVC signals

In [12]:
level = 4
arrNum = 0
healthyDetails = [pywt.wavedec(signal, "db4", level = level)[arrNum] for signal in filteredHealthy]
PVCDetails = [pywt.wavedec(signal, "db4", level = level)[arrNum] for signal in filteredPVC]
healthyTestDetails = [pywt.wavedec(signal, "db4", level = level)[arrNum] for signal in filteredHealtyTest]
PVCTestDetails = [pywt.wavedec(signal, "db4", level = level)[arrNum] for signal in filteredPVCTest]
#draw(pywt.wavedec(filteredHealthy[0], "db4", level = level)[0])

In [13]:
draw(healthyDetails[0])

In [14]:
draw(PVCDetails[0])

In [15]:
draw(healthyTestDetails[0])

### We can see that the difference between them is noticeable 

### We will use min-max scaling so the signal would range from [0,1]

In [16]:
def normalize(signal):
    return (signal - np.min(signal)) / (np.max(signal) - np.min(signal))

In [17]:
normalizedHealthy = [normalize(signal) for signal in healthyDetails]
normalizedPVC = [normalize(signal) for signal in PVCDetails]
normalizedHealthyTest = [normalize(signal) for signal in healthyTestDetails]
normalizedPVCTest = [normalize(signal) for signal in PVCTestDetails]

### A small comparison between processed signal vs raw signal

In [18]:
draw(PVCSignals[0])

In [19]:
draw(normalizedPVC[0])

* PVC : 1
* Healthy : 0

In [20]:
### Creating the DF and shuffling so its ready for training

In [21]:
data = normalizedHealthy + normalizedPVC
labels = list(np.zeros(200)) + list(np.ones(200))

df = pd.DataFrame({"x": data, "y": labels})
df = shuffle(df).reset_index(drop=True)

In [22]:
df

,x,y
0,"[0.4945083113217142, 0.4943360419492853, 0.495...",1.0
1,"[0.6415133589793973, 0.6409191036926074, 0.639...",1.0
2,"[0.16290192140876494, 0.16352577634882828, 0.1...",0.0
3,"[0.26931584613115545, 0.26868680112275944, 0.2...",0.0
4,"[0.25663961710197647, 0.2549500063159101, 0.25...",0.0
...,...,...
395,"[0.14925839824502563, 0.14875048082925205, 0.1...",0.0
396,"[0.2835541694752079, 0.2826391571630899, 0.281...",0.0
397,"[0.26880705814693906, 0.25932472252920186, 0.2...",0.0
398,"[0.46536525363994613, 0.46522109923526656, 0.4...",1.0


In [23]:
### Creating the TestDF and shuffling so its ready for testing

In [24]:
testData = normalizedHealthyTest + normalizedPVCTest
testLabels = list(np.zeros(200)) + list(np.ones(200))

testDF = pd.DataFrame({"x": testData, "y": testLabels})
testDF = shuffle(df).reset_index(drop=True)

### Start training

In [25]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(list(df["x"]),list(df["y"]))

KNeighborsClassifier()

### Predict data and Measuring accuracy

In [26]:
yPredict = knn.predict(list(testDF['x']))
yPredict

array([0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 0., 0.

In [27]:
yTest = list(testDF['y'])
accuracy_score(yTest, yPredict)

0.99

In [28]:
print(classification_report(yTest, yPredict))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       200
         1.0       0.99      0.99      0.99       200

    accuracy                           0.99       400
   macro avg       0.99      0.99      0.99       400
weighted avg       0.99      0.99      0.99       400



In [29]:
def predict_signal_from_file(file_path, knn_model):

        input_signal = readHB(file_path)
        filtered_signal = signal.lfilter(b, a, input_signal)
        print("before normalization")
        draw(filtered_signal[0])
        detail_coeffs = pywt.wavedec(filtered_signal, "db4",level=level)[0]
        normalized_signal = normalize(detail_coeffs)
        print("after normalization")
        draw(normalized_signal[0])
        normalized_signal
        prediction = knn_model.predict(normalized_signal)[0]

        return "Normal Heartbeat" if prediction == 0 else "PVC"
    


In [30]:
signalsDirPath = "/kaggle/input/signal-signals"
test_file_path = f"{signalsDirPath}/PVCECGSig_150.txt"
result = predict_signal_from_file(test_file_path, knn)
sigNumber = 8
# prediction = knn.predict(list(testDF['x'])[sigNumber])[0]

before normalization


after normalization


In [31]:
reality = 1
reality = "Normal Heartbeat" if reality == 0 else "PVC"
print(f"The {reality} signal in this file is classified as: {result}")

The PVC signal in this file is classified as: PVC
